# 박스만

In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
from ultralytics import YOLO  # YOLOv5 → YOLOv8 모델도 호환됨 (Ultralytics 통합됨)

def imread_unicode(filename):
    stream = np.fromfile(filename, dtype=np.uint8)
    return cv2.imdecode(stream, cv2.IMREAD_COLOR)

def imwrite_unicode(filename, img):
    ext = os.path.splitext(filename)[1]
    result, encoded_img = cv2.imencode(ext, img)
    if result:
        encoded_img.tofile(filename)
        return True
    return False

def load_yolo_model():
    return YOLO("yolov5s.pt")  # yolov5s, yolov5m, yolov5l 등 모델 변경 가능

def detect_all_person_boxes_with_padding(frame, model, padding_ratio=0.2):
    results = model.predict(source=frame, verbose=False)
    height, width = frame.shape[:2]
    boxes = []

    for r in results:
        for box in r.boxes:
            cls = int(box.cls)
            conf = float(box.conf)
            if conf > 0.5 and cls == 0:  # '0'은 COCO의 'person' 클래스
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                w = x2 - x1
                h = y2 - y1
                pad_w = int(w * (padding_ratio + 0.4))
                pad_h = int(h * padding_ratio)

                x = max(0, x1 - pad_w)
                y = max(0, y1 - pad_h)
                w = min(width - x, w + 2 * pad_w)
                h = min(height - y, h + 2 * pad_h)

                boxes.append((x, y, w, h))
    return boxes

def get_center(box):
    x, y, w, h = box
    return (x + w // 2, y + h // 2)

def get_image_center(img):
    h, w = img.shape[:2]
    return (w // 2, h // 2)

def euclidean_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_closest_box_to_point(boxes, point):
    min_dist = float('inf')
    best_box = None
    for box in boxes:
        center = get_center(box)
        dist = euclidean_distance(center, point)
        if dist < min_dist:
            min_dist = dist
            best_box = box
    return best_box

# YOLOv5 모델 불러오기
model = load_yolo_model()

# 경로 설정
input_dir = Path(r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\balanced_true\jpg")
output_dir = Path(r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\balanced_true\cropped")
output_dir.mkdir(parents=True, exist_ok=True)

# 추적 기준 박스
ref_box = None
distance_threshold = 150

for img_path in sorted(input_dir.glob("*.jpg")):
    save_path = output_dir / img_path.name
    if save_path.exists():
        continue

    img = imread_unicode(str(img_path))
    if img is None:
        print(f"[경고] 이미지 로드 실패: {img_path.name}")
        continue

    boxes = detect_all_person_boxes_with_padding(img, model)
    if not boxes:
        print(f"[경고] 사람 미검출: {img_path.name}")
        continue

    if ref_box is None:
        center_point = get_image_center(img)
        ref_box = get_closest_box_to_point(boxes, center_point)
    else:
        ref_center = get_center(ref_box)
        new_box = get_closest_box_to_point(boxes, ref_center)
        new_center = get_center(new_box)

        if euclidean_distance(ref_center, new_center) < distance_threshold:
            ref_box = new_box
        else:
            print(f"[경고] 추적 실패 (거리 초과): {img_path.name}")
            continue

    x, y, w, h = ref_box
    cropped = img[y:y+h, x:x+w]
    imwrite_unicode(str(save_path), cropped)


[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0001.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0002.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0003.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0004.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0005.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0006.jpg
[경고] 사람 미검출: 20201116_General_001_DOS_A_M40_MM_015_0007.jpg
[경고] 사람 미검출: 20201116_General_001_DOS_A_M40_MM_015_0008.jpg
[경고] 사람 미검출: 20201116_General_001_DOS_A_M40_MM_015_0009.jpg
[경고] 사람 미검출: 20201116_General_001_DOS_A_M40_MM_015_0010.jpg
[경고] 사람 미검출: 20201116_General_001_DOS_A_M40_MM_015_0011.jpg
[경고] 사람 미검출: 20201116_General_001_DOS_A_M40_MM_015_0012.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0013.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0014.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_001_DOS_A_M40_MM_015_0015.jpg
[경고] 추적 실패 (거리 초과): 20201116_General_

# 단일

In [12]:
import cv2
import numpy as np

def load_yolo_model():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

def detect_largest_person_box_with_padding(frame, net, output_layers, padding_ratio=0.1):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    largest_box = None
    largest_area = 0

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                pad_w = int(w * (padding_ratio+0.4))
                pad_h = int(h * padding_ratio)
                x = max(0, int(center_x - w / 2) - pad_w)
                y = max(0, int(center_y - h / 2) - pad_h)
                w = min(width - x, w + 2 * pad_w)
                h = min(height - y, h + 2 * pad_h)

                area = w * h
                if area > largest_area:
                    largest_area = area
                    largest_box = (x, y, w, h)

    return largest_box

# 경로 설정 및 실행
image_path = "1.jpg"  # <- 여기에 이미지 경로 지정
img = cv2.imread(image_path)
net, output_layers = load_yolo_model()

box = detect_largest_person_box_with_padding(img, net, output_layers, padding_ratio=0.2)

if box:
    x, y, w, h = box
    cropped = img[y:y+h, x:x+w]
    cv2.imshow("Cropped with Padding", cropped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("사람을 찾지 못했습니다.")
